In [8]:
import pandas as pd
import requests
import zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.display.float_format = '{:.4f}'.format

# Código completo na descrição

In [32]:
anos = 2
mes = []
ano = []

for i in range(anos * 12):
    mes.append((datetime.today()-relativedelta(months=i+1)).month)
    ano.append((datetime.today()-relativedelta(months=i+1)).year)

mes = mes[::-1]
ano = ano[::-1]

for i in range(len(mes)):
    if len(str(mes[i])) < 2:
        mes[i] = "0" + str(mes[i])
    else:
        mes[i] = str(mes[i])

for i in range(len(ano)):
    ano[i] = str(ano[i])

print(mes)
print(ano)

['03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '01', '02']
['2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023', '2023']


In [35]:
dados_fundos = pd.DataFrame()

for i in range(len(mes)):
    url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano[i]}{mes[i]}.zip'

    download = requests.get(url)

    with open(f"inf_diario_fi_{ano[i]}{mes[i]}.zip", "wb") as arquivo_cvm:

        arquivo_cvm.write(download.content) 
        
    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano[i]}{mes[i]}.zip")

    dados_fundos_mes = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

    dados_fundos = dados_fundos.append(dados_fundos_mes)

dados_fundos.shape

    

C:\Users\rmnf\AppData\Local\Temp\ipykernel_3508\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_3508\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_3508\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_3508\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(da

(11504387, 9)

In [36]:
dados_fundos.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
444198,FI,97.929.213/0001-34,2023-02-22,67257749.8900,10.3160,78394219.6100,0.0000,0.0000,2
444199,FI,97.929.213/0001-34,2023-02-23,77462678.4600,10.3171,78403192.7300,0.0000,0.0000,2
444200,FI,97.929.213/0001-34,2023-02-24,78120590.3500,10.3164,78397536.7100,0.0000,0.0000,2
444201,FI,97.929.213/0001-34,2023-02-27,68690770.1200,10.3256,78467221.6000,0.0000,0.0000,2
444202,FI,97.929.213/0001-34,2023-02-28,78214503.2500,10.3286,78490459.4900,0.0000,0.0000,2


In [37]:
dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                             sep = ";", encoding = 'ISO-8859-1')

dados_cadastro = dados_cadastro[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

dados_cadastro = dados_cadastro.drop_duplicates()

dados_cadastro

C:\Users\rmnf\AppData\Local\Temp\ipykernel_3508\3548788005.py:1: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',


,CNPJ_FUNDO,DENOM_SOCIAL
0,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCE...
1,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INV...
2,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO
3,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC ...
4,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIME...
...,...,...
71321,28.850.140/0001-78,FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGR...
71322,28.912.043/0001-62,HAPVIDA - FUNDO DE FINANCIAMENTO DA INDUSTRIA ...
71323,31.674.280/0001-47,MEDIA VENTURE ESG INCENTIVADO - FUNDO DE FINAN...
71324,32.222.962/0001-81,Fundo de Financiamento da Indústria Cinematogr...


In [39]:
data_inicio = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[0]
data_fim = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-1]
        

dados_fundos_filtrado = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio, data_fim]))]       

dados_fundos_filtrado.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
444130,FI,97.548.164/0001-90,2023-02-28,59551420.8400,0.1519,59751584.0700,0.0000,0.0000,1
444148,FI,97.548.167/0001-23,2023-02-28,5337680974.2100,4.4325,5126409391.8300,2852045.0000,0.0000,12
444166,FI,97.711.801/0001-05,2023-02-28,86573287.2100,2.8943,86545827.3600,0.0000,0.0000,1
444184,FI,97.929.197/0001-80,2023-02-28,132053514.5000,4.7015,132211376.8600,0.0000,0.0000,13
444202,FI,97.929.213/0001-34,2023-02-28,78214503.2500,10.3286,78490459.4900,0.0000,0.0000,2


In [40]:
dados_fundos_filtrado.head()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-03-01,1090537.2200,27.4902,1088117.6900,0.0000,0.0000,1
23,FI,00.068.305/0001-35,2021-03-01,43697116.2200,27.4660,43680005.5700,0.0000,1927.0700,7126
46,FI,00.071.477/0001-68,2021-03-01,31012275987.1700,10.0297,31009924790.2000,2268592360.5500,2491517533.0100,300673
69,FI,00.073.041/0001-08,2021-03-01,17626333.1700,28.8443,17620825.1900,0.0000,0.0000,938
92,FI,00.083.181/0001-67,2021-03-01,16353488063.1800,701.3233,16344247653.9000,0.0000,0.0000,4


In [41]:
dados_fundos_filtrado.shape

(44529, 9)

In [42]:
base_final = pd.merge(dados_fundos_filtrado, dados_cadastro, how = "left", 
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final = base_final[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

base_final

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
0,00.017.024/0001-53,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL,2021-03-01,27.4902,1088117.6900,1
1,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2021-03-01,27.4660,43680005.5700,7126
2,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2021-03-01,10.0297,31009924790.2000,300673
3,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2021-03-01,28.8443,17620825.1900,938
4,00.083.181/0001-67,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...,2021-03-01,701.3233,16344247653.9000,4
...,...,...,...,...,...,...
44528,97.548.164/0001-90,DYNAMIC II - FUNDO DE INVESTIMENTO MULTIMERCAD...,2023-02-28,0.1519,59751584.0700,1
44529,97.548.167/0001-23,JGP STRATEGY MASTER FUNDO DE INVESTIMENTO MULT...,2023-02-28,4.4325,5126409391.8300,12
44530,97.711.801/0001-05,953 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS D...,2023-02-28,2.8943,86545827.3600,1
44531,97.929.197/0001-80,LECT FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,2023-02-28,4.7015,132211376.8600,13


In [43]:
fundos_prev = base_final[base_final['DENOM_SOCIAL'].str.contains("PREV", na = False)]
fundos_prev = fundos_prev[fundos_prev.VL_PATRIM_LIQ >= 10000000000]
#fundos_prev = fundos_prev[fundos_prev.NR_COTST == 1]
fundos_prev = fundos_prev.sort_values(by="VL_PATRIM_LIQ", ascending=False)

fundos_prev

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
21717,07.593.972/0001-86,BRASILPREV TOP TPF FUNDO DE INVESTIMENTO RENDA...,2023-02-28,5.4428,141106069277.0200,275
1992,07.593.972/0001-86,BRASILPREV TOP TPF FUNDO DE INVESTIMENTO RENDA...,2021-03-01,4.5232,92348003659.4700,224
653,03.737.219/0001-66,FUNDO DE INVESTIMENTO PREVIDENCIÁRIO CAIXA PRE...,2021-03-01,4.7119,76735456619.2100,21
1828,07.103.364/0001-46,ITAÚ FLEXPREV RENDA FIXA - FUNDO DE INVESTIMENTO,2021-03-01,4.6805,63843776559.7900,98
1860,07.187.548/0001-31,BRADESCO FUNDO DE INVESTIMENTO RENDA FIXA MÁST...,2021-03-01,4.6451,62621097602.3000,30
...,...,...,...,...,...,...
26023,17.999.961/0001-54,BRADESCO FUNDO DE INVESTIMENTO RENDA FIXA MÁST...,2023-02-28,2.2639,10433563248.8700,5
657,03.737.224/0001-79,FUNDO DE INV EM COTAS DE FUNDOS DE INV PREVIDE...,2021-03-01,5.7588,10302112204.8800,1
3820,11.060.657/0001-60,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2021-03-01,1.8732,10212542888.8600,1
29885,27.884.294/0001-18,ITAÚ FLEXPREV VÉRTICE RENDA FIXA VISION FUNDO ...,2023-02-28,15.4390,10207761651.3900,2


In [45]:
fundos_prev["DT_COMPTC"].iloc[0]

'2023-02-28'

# Patrimônio e performance do fundo em dezembro

In [46]:
maior_fundo = fundos_prev[(fundos_prev['CNPJ_FUNDO'] == fundos_prev["CNPJ_FUNDO"].iloc[0]) &
                                          ( fundos_prev['DT_COMPTC'] == fundos_prev["DT_COMPTC"].iloc[0])]

patrimonio_do_fundo =  "R$ " + str((maior_fundo['VL_PATRIM_LIQ'].iloc[0]/1000000000).round(2)) + " Bi"

patrimonio_do_fundo

'R$ 141.11 Bi'

In [47]:
fundos_prev[(fundos_prev['CNPJ_FUNDO'] == fundos_prev["CNPJ_FUNDO"].iloc[0])]

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
21717,07.593.972/0001-86,BRASILPREV TOP TPF FUNDO DE INVESTIMENTO RENDA...,2023-02-28,5.4428,141106069277.0200,275
1992,07.593.972/0001-86,BRASILPREV TOP TPF FUNDO DE INVESTIMENTO RENDA...,2021-03-01,4.5232,92348003659.4700,224


In [48]:
maior_fundo_periodo = fundos_prev[(fundos_prev['CNPJ_FUNDO'] == fundos_prev["CNPJ_FUNDO"].iloc[0])]
                                          
retorno = ("Retorno fundo: " +
            str(((maior_fundo_periodo['VL_QUOTA'].iloc[0]/maior_fundo_periodo['VL_QUOTA'].iloc[-1] - 1)
                 * 100).round(2)) + "%") 
    
retorno

'Retorno fundo: 20.33%'

In [49]:
!pip install quandl

     ---------------------------------------- 0.0/54.2 kB ? eta -:--:--
     ---------------------------------------- 54.2/54.2 kB 2.9 MB/s eta 0:00:00


In [56]:
import quandl

quandl.ApiConfig.api_key = "YUSDTrHyCjE7Gd8dP_yn"

ipca_mes = quandl.get("BCB/433", start_date = pd.to_datetime(maior_fundo_periodo['VL_QUOTA'].iloc[-1]))
cdi_mes = quandl.get("BCB/4391", start_date = pd.to_datetime(maior_fundo_periodo['VL_QUOTA'].iloc[-1]))

Timestamp('1970-01-01 00:00:00.000000004')